## CASE STUDY - Deploying a recommender

We have seen the movie lens data on a toy dataset now lets try something a little bigger.  You have some
choices.

* [MovieLens Downloads](https://grouplens.org/datasets/movielens/latest/)

If your resources are limited (your working on a computer with limited amount of memory)

> continue to use the sample_movielens_ranting.csv

If you have a computer with at least 8GB of RAM

> download the ml-latest-small.zip

If you have the computational resources (access to Spark cluster or high-memory machine)

> download the ml-latest.zip

The two important pages for documentation are below.

* [Spark MLlib collaborative filtering docs](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html) 
* [Spark ALS docs](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS)


In [1]:
import os
import shutil
import pandas as pd
import numpy as np
import pyspark as ps
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.types import DoubleType

In [2]:
## ensure the spark context is available
spark = (ps.sql.SparkSession.builder
        .appName("sandbox")
        .getOrCreate()
        )

sc = spark.sparkContext
print(spark.version) 

2.4.5


### Ensure the data are downloaded and specify the file paths here


In [3]:
data_dir = os.path.join(".", "data")
ratings_file = os.path.join(data_dir, "ratings.csv")
movies_file = os.path.join(data_dir, "movies.csv")

In [4]:
# Load the data
ratings = spark.read.csv(ratings_file, header=True, inferSchema=True)
movies = spark.read.csv(movies_file, header=True, inferSchema=True)

ratings.show(n=4)
movies.show(n=4)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
+------+-------+------+---------+
only showing top 4 rows

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
+-------+--------------------+--------------------+
only showing top 4 rows



## QUESTION 1

Explore the movie lens data a little and summarize it

In [5]:
## (summarize the data)

## rename columns
ratings = ratings.withColumnRenamed("movieID", "movie_id")
ratings = ratings.withColumnRenamed("userID", "user_id")

ratings.describe().show()

print("Unique users {}".format(ratings.select("user_id").distinct().count()))
print("Unique movies {}".format(ratings.select("movie_id").distinct().count()))
print('Movies with Rating > 2: {}'.format(ratings.filter('rating > 2').select('movie_id').distinct().count()))
print('Movies with Rating > 3: {}'.format(ratings.filter('rating > 3').select('movie_id').distinct().count()))
print('Movies with Rating > 4: {}'.format(ratings.filter('rating > 4').select('movie_id').distinct().count()))

+-------+------------------+----------------+------------------+--------------------+
|summary|           user_id|        movie_id|            rating|           timestamp|
+-------+------------------+----------------+------------------+--------------------+
|  count|            100836|          100836|            100836|              100836|
|   mean|326.12756356856676|19435.2957177992| 3.501556983616962|1.2059460873684695E9|
| stddev| 182.6184914635004|35530.9871987003|1.0425292390606342|2.1626103599513078E8|
|    min|                 1|               1|               0.5|           828124615|
|    max|               610|          193609|               5.0|          1537799250|
+-------+------------------+----------------+------------------+--------------------+

Unique users 610
Unique movies 9724
Movies with Rating > 2: 8852
Movies with Rating > 3: 7363
Movies with Rating > 4: 4056


## QUESTION 2

Find the ten most popular movies---that is the then movies with the highest average rating

>Hint: you may want to subset the movie matrix to only consider movies with a minimum number of ratings

In [6]:
## Group movies
movies_counts = ratings.groupBy("movie_id").count()
movies_rating = ratings.groupBy("movie_id").avg("rating")
movies_rating_and_count = movies_counts.join(movies_rating, "movie_id")

## Consider movies with more than 100 views
threshold = 100
top_movies =  movies_rating_and_count.filter("count > 100").orderBy("avg(rating)", ascending=False)

## Add the movie titles to data frame
movies = movies.withColumnRenamed("movieID", "movie_id")
top_movies = top_movies.join(movies, "movie_id")

top_movies.toPandas().head(10)

,movie_id,count,avg(rating),title,genres
0,318,317,4.429022,"Shawshank Redemption, The (1994)",Crime|Drama
1,858,192,4.289062,"Godfather, The (1972)",Crime|Drama
2,2959,218,4.272936,Fight Club (1999),Action|Crime|Drama|Thriller
3,1221,129,4.259690,"Godfather: Part II, The (1974)",Crime|Drama
4,48516,107,4.252336,"Departed, The (2006)",Crime|Drama|Thriller
5,1213,126,4.250000,Goodfellas (1990),Crime|Drama
6,58559,149,4.238255,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX
7,50,204,4.237745,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
8,1197,142,4.232394,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance
9,260,251,4.231076,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi


## QUESTION 3

Compare at least 5 different values for the ``regParam``

Use the `` ALS.trainImplicit()`` and compare it to the ``.fit()`` method.  See the [Spark ALS docs](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS)
for example usage. 

In [7]:
## split the data set to  train and test set

(train, test) = ratings.randomSplit([0.8, 0.2])

## Create a function to train the model
def train_model(reg_param, implicit_prefs=False):
    als = ALS(maxIter=5, regParam=reg_param, userCol="user_id", itemCol="movie_id", ratingCol="rating", coldStartStrategy="drop", implicitPrefs=implicit_prefs)
    model = als.fit(train)

    predictions = model.transform(test)
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

    rmse = evaluator.evaluate(predictions)
    print("regParam={}, RMSE={}".format(reg_param, np.round(rmse,2)))
    

for reg_param in [0.01, 0.05, 0.1, 0.15, 0.25]:
    train_model(reg_param)

regParam=0.01, RMSE=1.08
regParam=0.05, RMSE=0.95
regParam=0.1, RMSE=0.89
regParam=0.15, RMSE=0.88
regParam=0.25, RMSE=0.91


## QUESTION 4

With your best `regParam` try using the `implicitPrefs` flag.

In [8]:
train_model(reg_param=0.1, implicit_prefs=True)

regParam=0.1, RMSE=3.24


## QUESTION 5

Use model persistence to save your finalized model

In [9]:
## re-train using the whole data set
print("...training")
als = ALS(maxIter=5, regParam=0.1, userCol="user_id", itemCol="movie_id", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(ratings)

## save the model for furture use
save_dir = "./models/saved-recommender"
if os.path.isdir(save_dir):
    print("...overwritting saved model")
    shutil.rmtree(save_dir)

## save the top-ten movies
print("...saving top-movies")
top_movies.toPandas().head(10).to_csv("./data/top-movies.csv", index=False)
    
## save model
model.save(save_dir)
print("done.")

...training
...overwritting saved model
...saving top-movies
done.


## QUESTION 6

Use ``spark-submit`` to load the model and demonstrate that you can load the model and interface with it.

In [10]:
from pyspark.ml.recommendation import ALSModel
from_saved_model = ALSModel.load(save_dir)

In [11]:
test = spark.createDataFrame([(1, 5), (1, 10), (2, 1)], ["user_id", "movie_id"])
predictions = sorted(model.transform(test).collect(), key=lambda r: r[0])
print(predictions)

[Row(user_id=1, movie_id=5, prediction=3.4502124786376953), Row(user_id=1, movie_id=10, prediction=3.862915515899658), Row(user_id=2, movie_id=1, prediction=3.4418039321899414)]


In [12]:
%%writefile ./scripts/case-study-spark-submit.sh

#!/bin/bash
${SPARK_HOME}/bin/spark-submit \
--master local[4] \
--executor-memory 1G \
--driver-memory 1G \
$@

Overwriting ./scripts/case-study-spark-submit.sh


In [13]:
!chmod 711 ./scripts/case-study-spark-submit.sh

In [14]:
! ./scripts/case-study-spark-submit.sh ./scripts/recommender-submit.py

20/04/25 13:02:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
20/04/25 13:02:14 INFO SparkContext: Running Spark version 2.4.5
20/04/25 13:02:14 INFO SparkContext: Submitted application: recommend
20/04/25 13:02:14 INFO SecurityManager: Changing view acls to: jovyan
20/04/25 13:02:14 INFO SecurityManager: Changing modify acls to: jovyan
20/04/25 13:02:14 INFO SecurityManager: Changing view acls groups to: 
20/04/25 13:02:14 INFO SecurityManager: Changing modify acls groups to: 
20/04/25 13:02:14 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(jovyan); groups with view permissions: Set(); users  with modify permissions: Set(jovyan); groups with modify permissions: Set()
20/04/25 13:02:15 INFO Utils: Successfully started service 'sparkDriver' on port 34689.
20/04